<a href="https://colab.research.google.com/github/nhanlunlk2k/APP-project/blob/main/app_group7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import copy

In [21]:
def find_min_max_2D(M):
    min = M[0][0]
    max = M[0][0]
    for i in range(len(M)):
        for j in range(len(M[i])):
            if M[i][j] < min:
                min = M[i][j]
            elif M[i][j] > max:
                max = M[i][j]
    return min,max

def find_mean_2D(M):
    sum = 0
    for i in range(len(M)):
        for j in range(len(M[i])):
            sum += M[i][j]
    return sum/(len(M)*len(M[0]))

def find_median_2D(M):
    tmp = copy.copy(M[0])
    for i in range(1,len(M)):
        tmp.extend(M[i])
    tmp.sort()
    if len(tmp)%2 == 0:
        return (tmp[len(tmp)//2] + tmp[len(tmp)//2 - 1])/2
    else:
        return tmp[len(tmp)//2]

def pad_2D(M,padding_size,mode):
    num_cols = len(M[0]) + padding_size[0][0] + padding_size[0][1]
    #num_rows = len(M) + padding_size[1][0] + padding_size[1][1]
    match mode:
        case 'min':
            cons = find_min_max_2D(M)[0]
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,cons)
                for j in range(padding_size[0][1]):
                    M[i].append(cons)
            line = [cons]*num_cols
            for j in range(padding_size[1][0]):
                M.insert(0,line)
            for j in range(padding_size[1][1]):
                M.append(line)
            return M
        case 'max':
            cons = find_min_max_2D(M)[1]
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,cons)
                for j in range(padding_size[0][1]):
                    M[i].append(cons)
            line = [cons]*num_cols
            for j in range(padding_size[1][0]):
                M.insert(0,line)
            for j in range(padding_size[1][1]):
                M.append(line)
            return M
        case 'mean':
            cons = find_mean_2D(M)
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,cons)
                for j in range(padding_size[0][1]):
                    M[i].append(cons)
            for j in range(padding_size[1][0]):
                M.insert(0,[cons]*num_cols)
            for j in range(padding_size[1][1]):
                M.append([cons]*num_cols)
            return M
        case 'zero':
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,0)
                for j in range(padding_size[0][1]):
                    M[i].append(0)
            for j in range(padding_size[1][0]):
                M.insert(0,[0]*num_cols)
            for j in range(padding_size[1][1]):
                M.append([0]*num_cols)
            return M
        case 'median':
            cons = find_median_2D(M)
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,cons)
                for j in range(padding_size[0][1]):
                    M[i].append(cons)
            for j in range(padding_size[1][0]):
                M.insert(0,[cons]*num_cols)
            for j in range(padding_size[1][1]):
                M.append([cons]*num_cols)
            return M
        case 'edge':
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,M[i][0])
        #print(M)
                for j in range(padding_size[0][1]):
                    M[i].append(M[i][-1])
        #print(M)
            M = [M[0]]*padding_size[1][0] + M + [M[-1]]*padding_size[1][1]
    #print(M)
            return M
        case _:
            raise ValueError('Invalid mode')
    pass

In [76]:
def num_dim(A):
    if type(A) == list:
        return 1 + num_dim(A[0])
    else:
        return 0

def array_shape(A):
    shape = []
    while type(A) == list:
        shape.append(len(A))
        A = A[0]
    return shape

def small_convolution(A,B): #3D convolution with same size
    res = 0
    for k in range(len(A)):
        for i in range(len(A[k])):
            for j in range(len(A[k][i])):
                res += A[k][i][j]*B[k][i][j]
    return res

def extract_array(A,begin,end):
    res = []
    for i in range(begin[0],end[0]):
        line = []
        for j in range(begin[1],end[1]):
            line.append(A[i][j])
        res.append(line)
    return res

def convolution1_py(F,M,padding=None): #F is a filter (2d array), M is a volume (3d matrix)
    up_padding = len(F)//2
    down_padding = len(F) - len(F)//2 - 1
    left_padding = len(F[0])//2
    right_padding = len(F) - len(F[0])//2 - 1
    F = [F]*len(M)
    pad_size = ((left_padding,right_padding),(up_padding,down_padding))
    res = []
    if padding != None:
        for i in range(len(M)):
            M[i] = pad_2D(M[i],pad_size,padding)
        res = [[0]*len(M[0][0])]*len(M[0])
    else:
        res = [[0]*(len(M[0][0]) - len(F) + 1)]*(len(M[0]) - len(F[0]) + 1)
    for i in range(len(res)):
        for j in range(len(res[i])):
            res[i][j] = small_convolution(extract_array(M, (i,j), (i+len(F),j+len(F[0]))),F)
    return res

In [4]:
def convolution_np(F,M): #F is a matrix (2d array), M is a volume (3d matrix)
    pass

In [77]:
a = [[6,1,8],[3,2,5],[9,5,7],[4,3,6]]
#tmp = a[0]
#for i in range(1,len(a)):
#    tmp += a[i]
#tmp.sort()
#print(tmp)
#tmp.sort()
#tmp
array_shape(a)
#c=find_median_2D(a)
#for i in range(len(a)):
#    for j in range(2):
#        a[i].insert(0,c)
extract_array(a,(1,1),(3,3))

[[2, 5], [5, 7]]

In [65]:
[[]]*6

[[], [], [], [], [], []]